### Setup

In [1]:
import datetime

In [2]:
import os
pwd = os.path.abspath('.')
os.chdir(os.path.join(pwd, '../src/'))

In [3]:
%load_ext autoreload
%autoreload 2

  ### Get Index data for date range

In [4]:
from markets_insights.datareader import data_reader
import datetime

reader = data_reader.NseIndicesReader()

daterange_reader = data_reader.DateRangeDataReader(reader)

from_date = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
result = daterange_reader.read(data_reader.DateRangeCriteria(from_date, to_date))
print(result.head(3).to_markdown())

c:\Users\NRohra\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


|    | Identifier    | Index Date   |     Open |    High |     Low |    Close |   Points Change |   Change(%) |      Volume |    Turnover |   P/E |   P/B |   Div Yield | Date                |
|---:|:--------------|:-------------|---------:|--------:|--------:|---------:|----------------:|------------:|------------:|------------:|------:|------:|------------:|:--------------------|
|  0 | Nifty 50      | 02-01-2023   | 18131.7  | 18215.2 | 18086.5 | 18197.5  |           92.15 |        0.51 | 2.56074e+08 | 1.17931e+11 | 21.9  |  4.27 |        1.26 | 2023-01-02 00:00:00 |
|  1 | Nifty Next 50 | 02-01-2023   | 42321.1  | 42409.8 | 42117.7 | 42248.4  |           60.75 |        0.14 | 1.34929e+08 | 4.11122e+10 | 25.67 |  4.79 |        1.76 | 2023-01-02 00:00:00 |
|  2 | Nifty 100     | 02-01-2023   | 18290.2  | 18350.9 | 18237.5 | 18334.3  |           75.55 |        0.41 | 3.91003e+08 | 1.59043e+11 | 22.75 |  4.41 |        1.3  | 2023-01-02 00:00:00 |
|  3 | Nifty 200     | 02-01-2023   |  9

### Get daily, monthly and annually aggregrated data
In this example we will use HistoricalDataProcessor class to get data between a date range. HistoricalDataProcessor will also do monthly and annual aggregation of data.

In [5]:
# import classes & setup
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor
from markets_insights.datareader.data_reader import NseIndicesReader, DateRangeCriteria
histDataProcessor = HistoricalDataProcessor()

# Fetch and process the data
year_start = datetime.date(2023, 12, 1)
year_end = datetime.date(2023, 12, 31)
result = histDataProcessor.process(NseIndicesReader(), DateRangeCriteria(year_start, year_end))

Started to read data
HistoricalDataProcessor.get_data took 0 seconds
HistoricalDataProcessor.get_manual_data took 0 seconds
HistoricalDataProcessor.run_base_calculations took 0 seconds
Started periodic calculation for Month
HistoricalDataProcessor.add_monthly_growth_calc took 0 seconds
Started periodic calculation for Year
HistoricalDataProcessor.add_yearly_growth_calc took 0 seconds


c:\Data\_Personal\Projects\markets_insights\src\markets_insights\dataprocess\data_processor.py:329: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([historical_data, manual_data], ignore_index=True)


HistoricalDataProcessor.process took 1 seconds


In [6]:
from markets_insights.core.column_definition import BaseColumns

print(result.get_monthly_data().sort_values(BaseColumns.Date).head(3).to_markdown())

|    | Identifier                             | Date                | Month   |      Volume |    Turnover |   Close |     High |     Low |     Open |
|---:|:---------------------------------------|:--------------------|:--------|------------:|------------:|--------:|---------:|--------:|---------:|
|  0 | INDIA VIX                              | 2023-12-29 00:00:00 | 2023-12 | 0           | 0           |    14.5 |    16.47 |    10.8 |    12.69 |
| 78 | NIFTY SMALLCAP250 MOMENTUM QUALITY 100 | 2023-12-29 00:00:00 | 2023-12 | 2.15706e+09 | 5.50912e+11 | 41504   | 41504    | 40304.2 | 40304.2  |
| 77 | NIFTY SMALLCAP 50                      | 2023-12-29 00:00:00 | 2023-12 | 7.58478e+09 | 1.22615e+12 |  7132.1 |  7145.2  |  6566.2 |  6571    |


### Calculation pipeline for RSI

In [7]:
import datetime
from markets_insights.datareader.data_reader import BhavCopyReader, DateRangeCriteria
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor, MultiDataCalculationPipelines, CalculationPipelineBuilder, HistoricalDataProcessOptions
from markets_insights.calculations.base import DatePartsCalculationWorker

reader = BhavCopyReader()
options = HistoricalDataProcessOptions()
options.include_monthly_data = False
options.include_annual_data = False
histDataProcessor = HistoricalDataProcessor(options)

# Fetch the data
year_start = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
result = histDataProcessor.process(reader, DateRangeCriteria(year_start, to_date))

# Prepare calculation pipeline
pipelines = MultiDataCalculationPipelines()
pipelines.set_item('rsi', CalculationPipelineBuilder.create_rsi_calculation_pipeline())
histDataProcessor.set_calculation_pipelines(pipelines)

# Run the pipeline
histDataProcessor.run_calculation_pipelines()

Started to read data
HistoricalDataProcessor.get_data took 1 seconds
HistoricalDataProcessor.get_manual_data took 0 seconds
HistoricalDataProcessor.run_base_calculations took 17 seconds
HistoricalDataProcessor.process took 18 seconds


c:\Data\_Personal\Projects\markets_insights\src\markets_insights\calculations\base.py:385: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = data.groupby(self.get_group_cols(data.columns), group_keys=True).apply(


RsiCalculationWorker took 8 seconds
ColumnValueCrossedAboveFlagWorker took 1 seconds
ColumnValueCrossedBelowFlagWorker took 1 seconds


In [8]:
from markets_insights.core.column_definition import BaseColumns, CalculatedColumns

print(result.get_daily_data().sort_values(BaseColumns.Date).tail(3)[[BaseColumns.Identifier, BaseColumns.Date, BaseColumns.Close, CalculatedColumns.RelativeStrengthIndex]].to_markdown())

|        | Identifier   | Date                |   Close |     Rsi |
|-------:|:-------------|:--------------------|--------:|--------:|
| 148631 | GULFOILLUB   | 2023-12-29 00:00:00 |  718.45 | 62.6257 |
| 326542 | RAMASTEEL    | 2023-12-29 00:00:00 |   36    | 40.4125 |
| 446904 | ZYDUSWELL    | 2023-12-29 00:00:00 | 1681.1  | 72.0326 |


### Test Calculation

In [9]:
from markets_insights.core.column_definition import BaseColumns
from markets_insights.calculations.base import SmaCalculationWorker, \
  ColumnValueAboveAnotherColumnValueFlagWorker
from markets_insights.core.core import FlagFilter
from markets_insights.datareader.data_reader import BhavCopyReader, DateRangeCriteria
from markets_insights.dataprocess.data_processor import HistoricalDataset, HistoricalDataProcessor, HistoricalDataProcessOptions, \
  CalculationPipelineBuilder, MultiDataCalculationPipelines

from datetime import date

processor = HistoricalDataProcessor(HistoricalDataProcessOptions(include_annual_data=False, include_monthly_data=False))
result: HistoricalDataset = processor.process(
    BhavCopyReader(),#.set_filter(IdentifierFilter("INFY")),
    DateRangeCriteria(date(2023, 10, 1), date(2023, 12, 31)),
)
result._daily.drop_duplicates(inplace=True)

workers = [
  SmaCalculationWorker(time_window=50),
  ColumnValueAboveAnotherColumnValueFlagWorker(value_column_a=BaseColumns.Close, value_column_b='Sma50')
]

# Prepare calculation pipeline
pipelines = MultiDataCalculationPipelines()
pipelines.set_item(
    "user", CalculationPipelineBuilder.create_pipeline_for_workers(workers)
)
processor.set_calculation_pipelines(pipelines)

# Run the pipeline and get data
processor.run_calculation_pipelines()
daily_data = result.get_daily_data()
data = daily_data.query(str(FlagFilter(workers[1].get_columns()[0])))
data[['Identifier', 'Date', 'Sma50', workers[1].get_columns()[0]]]


Started to read data
HistoricalDataProcessor.get_data took 1 seconds
HistoricalDataProcessor.get_manual_data took 0 seconds
HistoricalDataProcessor.run_base_calculations took 4 seconds
HistoricalDataProcessor.process took 5 seconds
SmaCalculationWorker took 0 seconds
ColumnValueAboveAnotherColumnValueFlagWorker took 0 seconds


,Identifier,Date,Sma50,CloseAboveSma50
87401,CHEMPLASTS,2023-12-14,461.192,True
87404,AARTIPHARM,2023-12-14,417.221,True
87406,360ONE,2023-12-14,555.223,True
87407,ABSLAMC,2023-12-14,448.659,True
87408,AARTISURF,2023-12-14,615.411,True
...,...,...,...,...
107288,ZUARI,2023-12-29,165.276,True
107289,ZENSARTECH,2023-12-29,533.377,True
107290,WSTCSTPAPR,2023-12-29,706.422,True
107292,VPRPL,2023-12-29,204.410,True


In [8]:
daily_data[daily_data[workers[1].get_columns()[0]] == False][['Identifier', 'Date', 'Sma50', workers[1].get_columns()[0]]]

,Identifier,Date,Sma50,CloseAboveSma50
0,OSWALAGRO,2023-10-03,NaN,False
1,ORIENTPPR,2023-10-03,NaN,False
2,PAKKA,2023-10-03,NaN,False
3,PAISALO,2023-10-03,NaN,False
4,ORIENTHOT,2023-10-03,NaN,False
...,...,...,...,...
109019,YATHARTH,2023-12-29,382.476,False
109026,ZEEMEDIA,2023-12-29,15.184,False
109029,ZIMLAB,2023-12-29,116.195,False
109030,ZODIACLOTH,2023-12-29,132.779,False


In [15]:
data[['Identifier', 'Date', workers[1].get_columns()[0]]].tail(10)

,Identifier,Date,CloseCrossedBelowSma50
446701,STERTOOLS,2023-12-28,True
446743,SHAKTIPUMP,2023-12-28,True
446790,AAVAS,2023-12-28,True
446810,ABFRL,2023-12-28,True
446814,AGARIND,2023-12-28,True
446835,AMBER,2023-12-28,True
446866,BALKRISIND,2023-12-28,True
446915,CMSINFO,2023-12-28,True
446956,ASTRAL,2023-12-28,True
446980,ABSLLIQUID,2023-12-28,True


In [63]:
from markets_insights.core.core import IdentifierFilter, InstrumentTypeFilter
from markets_insights.datareader.data_reader import BhavCopyReader, NseDerivatiesReader


from datetime import date

cash_reader = BhavCopyReader().set_filter(IdentifierFilter("INDIACEM"))
filters = IdentifierFilter("INDIACEM") & InstrumentTypeFilter('FUTSTK')
fut_reader = NseDerivatiesReader().set_filter(filters)

premium_reader = cash_reader - fut_reader

#filters.get_query()
fut_reader.read(date(2024, 2, 21))

,InstrumentType,Identifier,ExpiryDate,StrkPric,OptionType,PreviousClose,Open,High,Low,Close,...,LftmLw,UnitOfMeasr,TtlNbOfTxsExctd,Ccy,Rsvd01,Rsvd02,Rsvd03,Rsvd04,Rsvd05,Date
0,FUTSTK,INDIACEM,28-Mar-2024,0.0,XX,243.20,246.85,246.85,243.55,243.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-21
1,FUTSTK,INDIACEM,29-Feb-2024,0.0,XX,242.45,245.00,247.60,240.10,241.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-21
2,FUTSTK,INDIACEM,25-Apr-2024,0.0,XX,251.50,0.00,0.00,0.00,251.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-21


In [36]:
from markets_insights.core.column_definition import BaseColumns
import pandas as pd
import pandas_ta as pd_ta

data = pd.DataFrame(daily_data.tail(100))
data.index = data[BaseColumns.Date]
pd_ta.vwap(
  close=data[BaseColumns.Close],
  high=data[BaseColumns.High],
  low=data[BaseColumns.Low],
  volume=data[BaseColumns.Volume],
)

Date
2024-01-01     611.533333
2024-01-02     612.100000
2024-01-03     607.966667
2024-01-04     609.716667
2024-01-05     608.516667
2024-01-08     603.566667
2024-01-09     620.850000
2024-01-10     607.750000
2024-01-11     606.133333
2024-01-12     603.416667
2024-01-15     598.300000
2024-01-16     589.366667
2024-01-17     581.783333
2024-01-18     583.216667
2024-01-19     582.133333
2024-01-23     575.683333
2024-01-24     580.250000
2024-01-25     578.316667
2024-01-29     575.516667
2024-01-30     579.866667
2024-01-31     597.750000
2024-02-01     610.466667
2024-02-02     615.850000
2024-02-05     615.866667
2024-02-06     617.033333
2024-02-07     615.900000
2024-02-08     613.950000
2024-02-09     604.583333
2024-02-12     678.100000
2024-02-13     732.600000
2024-02-14     819.433333
2024-02-15     919.433333
2024-02-16     930.283333
2024-02-19     981.083333
2024-02-20    1007.583333
Name: VWAP_D, dtype: float64

In [16]:
from markets_insights.core.column_definition import BaseColumns
daily_data = result.get_daily_data()

if len(daily_data[BaseColumns.Identifier].unique()) > 1:
    data = daily_data.groupby(BaseColumns.Identifier)
    data.apply(
        lambda x: x[BaseColumns.Turnover].rolling(50).sum()
        / x.rolling(50)[BaseColumns.Volume].sum()
    ) \
    .reset_index(level=0, drop=True)
else:
    daily_data[BaseColumns.Turnover].rolling(50).sum() / daily_data[BaseColumns.Volume].rolling(50).sum()

In [17]:
daily_data[BaseColumns.Turnover].rolling(50).sum() / daily_data[BaseColumns.Volume].rolling(50).sum()

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
          ...     
241    2390.315815
242    2393.762796
243    2397.837564
244    2403.204247
245    2407.309569
Length: 246, dtype: float64

### A real use case: Understand the affect of RSI and Stochastic RSI on price
In this use case, understand the affect of RSI and Stochastic RSI on price

#### Preparing the data
We perform below steps to prepare our analysis data
- Calculate RSI for each day for all the stocks.
- Add a flag for whenever the RSI crosses the control limits (eg: above 75 and below 30)
- Calculate the highest and lowest price change in the next 1, 3, 5, 7 & 10 trading sessions.
- Find the median for highest price change and lowest price change whenever the RSI crosses the control limits.

In [11]:
# import classes
from markets_insights.datareader.data_reader import BhavCopyReader, DateRangeCriteria

# Fetch the data
year_start = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
result = histDataProcessor.process(BhavCopyReader(), DateRangeCriteria(year_start, to_date))

Started to read data
HistoricalDataProcessor.get_data took 1 seconds
HistoricalDataProcessor.get_manual_data took 0 seconds
HistoricalDataProcessor.run_base_calculations took 17 seconds
HistoricalDataProcessor.process took 18 seconds


In [12]:
# prepare calculation pipeline
periods = [1, 7, 15, 30, 45]

pipelines = MultiDataCalculationPipelines()
pipelines.set_item('rsi', CalculationPipelineBuilder.create_rsi_calculation_pipeline(crossing_above_flag_value = 75, crossing_below_flag_value = 30, window = 14))
pipelines.set_item('stoch_rsi', CalculationPipelineBuilder.create_stoch_rsi_calculation_pipeline(crossing_above_flag_value = 80, crossing_below_flag_value = 20, window = 14))
pipelines.set_item('foward_looking_fall', CalculationPipelineBuilder.create_forward_looking_price_fall_pipeline(periods))
pipelines.set_item('foward_looking_rise', CalculationPipelineBuilder.create_forward_looking_price_rise_pipeline(periods))
histDataProcessor.set_calculation_pipelines(pipelines=pipelines)

# run the pipeline and show results
histDataProcessor.run_calculation_pipelines()

daily_data = result.get_daily_data()

# Import constants so its easier to refer to column names
from markets_insights.core.column_definition import BaseColumns, CalculatedColumns

# get names of fwd looking price column names. Since, these column names are auto-generated there no constants for them
fwd_looking_price_fall_cols, fwd_looking_price_rise_cols = [x for x in daily_data.columns if 'HighestPercFallInNext' in x], \
    [x for x in daily_data.columns if 'HighestPercRiseInNext' in x]

c:\Data\_Personal\Projects\markets_insights\src\markets_insights\calculations\base.py:385: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = data.groupby(self.get_group_cols(data.columns), group_keys=True).apply(


RsiCalculationWorker took 8 seconds
ColumnValueCrossedAboveFlagWorker took 1 seconds
ColumnValueCrossedBelowFlagWorker took 1 seconds
StochRsiCalculationWorker took 11 seconds
ColumnValueCrossedAboveFlagWorker took 1 seconds
ColumnValueCrossedAboveFlagWorker took 1 seconds
ColumnValueCrossedBelowFlagWorker took 1 seconds
ColumnValueCrossedBelowFlagWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds


In [13]:
daily_data[
(daily_data[CalculatedColumns.RsiCrossedAbove])
][fwd_looking_price_fall_cols].median()

Series([], dtype: float64)

In [14]:
daily_data[
(daily_data[CalculatedColumns.RsiCrossedBelow])
][fwd_looking_price_rise_cols].median()

Series([], dtype: float64)

### Performing Arithmetic on Readers

In [16]:
from markets_insights.core.core import InstrumentTypeFilter
from markets_insights.datareader.data_reader import BhavCopyReader, NseDerivatiesReader, NseIndicesReader, ForDateCriteria
from markets_insights.core.core import IdentifierFilter

for_date = datetime.date(2023, 12, 5)

In [17]:
indices_reader = NseIndicesReader()
vix_reader = NseIndicesReader().set_filter(IdentifierFilter("India VIX"))
op_reader = indices_reader / vix_reader
data = op_reader.read(ForDateCriteria(for_date)).query(str(IdentifierFilter("Nifty 50 / India VIX")))

### Extended Data Reader for Nasdaq

In [19]:
import yfinance as yf
import pandas
from markets_insights.datareader.data_reader import DateRangeDataReader
from markets_insights.core.core import Instrumentation
from markets_insights.core.column_definition import BaseColumns

In [21]:
class NasdaqDataReader (DateRangeDataReader):
  def __init__(self, tickers: list = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META', 'TSLA', 'NVDA']):
    super().__init__(reader=None)
    self.tickers = tickers
    self.name = "NasdaqDataReader"

  @Instrumentation.trace(name="NasdaqDataReader.read")
  def read(self, criteria: DateRangeCriteria):
    df_list = list()
    for ticker in self.tickers:
        data = yf.download(ticker, group_by="Ticker", start=criteria.from_date, end=criteria.to_date)
        data['ticker'] = ticker
        df_list.append(data)

    # combine all dataframes into a single dataframe
    df = pandas.concat(df_list)

    final_data = df.reset_index().rename(columns = self.get_column_name_mappings())
    final_data[BaseColumns.Date] = pandas.to_datetime(final_data[BaseColumns.Date])
    return final_data
  
  def get_column_name_mappings(self):
    return {
      'ticker': BaseColumns.Identifier,
      'OPEN': BaseColumns.Open,
      'HIGH': BaseColumns.High,
      'LOW': BaseColumns.Low,
      'CLOSE': BaseColumns.Close
    }

In [23]:
# import classes & setup options
import datetime
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor, MultiDataCalculationPipelines, CalculationPipelineBuilder, HistoricalDataProcessOptions, DateRangeCriteria
from markets_insights.calculations.base import DatePartsCalculationWorker

reader = NasdaqDataReader()
options = HistoricalDataProcessOptions()
options.include_monthly_data = False
options.include_annual_data = False
histDataProcessor = HistoricalDataProcessor(options)

# Fetch the data
year_start = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
result = histDataProcessor.process(reader, DateRangeCriteria(year_start, to_date))

# Prepare calculation pipeline
pipelines = MultiDataCalculationPipelines()
pipelines.set_item('date_parts', CalculationPipelineBuilder.create_pipeline_for_worker(DatePartsCalculationWorker()))
pipelines.set_item('rsi', CalculationPipelineBuilder.create_rsi_calculation_pipeline())
histDataProcessor.set_calculation_pipelines(pipelines)

# Run the pipeline
histDataProcessor.run_calculation_pipelines()

Started to read data
Reading data from 2023-01-02 to 2023-12-29


Failed to get ticker 'AAPL' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)')))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAPL']: Exception('%ticker%: No timezone found, symbol may be delisted')
Failed to get ticker 'GOOGL' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)')))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['GOOGL']: Exception('%ticker%: No timezone found, symbol may be delisted')
Failed to get ticker 'MSFT' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caus

NasdaqDataReader.read took 3 seconds
Saving data to file: ../../_data/processed/historical/NasdaqDataReader.csv
HistoricalDataProcessor.get_data took 3 seconds
HistoricalDataProcessor.get_manual_data took 0 seconds
HistoricalDataProcessor.run_base_calculations took 0 seconds
HistoricalDataProcessor.process took 3 seconds
DatePartsCalculationWorker took 0 seconds
RsiCalculationWorker took 0 seconds


KeyError: 'Column not found: Rsi'

In [9]:
result.get_daily_data() \
  .sort_values(
    [BaseColumns.Date, BaseColumns.Identifier]
  )[
    [BaseColumns.Identifier, BaseColumns.Date, BaseColumns.Close, 
     CalculatedColumns.RelativeStrengthIndex]
  ] \
  .tail(5)

KeyError: "['PreviousClose', 'Turnover (Rs. Cr.)', 'TOTTRDQTY'] not in index"

### Create a Calculation Pipeline

In [24]:
## import modules
from markets_insights.calculations.base import CalculationWorker
from markets_insights.core.core import Instrumentation
from markets_insights.calculations.base import BaseColumns
import pandas

# Implement the worker class. The important aspect here is to override the add_calculated_columns() method
class FibonacciRetracementCalculationWorker (CalculationWorker):
  def __init__(self, time_window: int, level_perct: float):
    self._time_window = time_window
    self._level = level_perct / 100
    self._column_name = 'Fbr' + str(level_perct)

  @Instrumentation.trace(name="FibnocciRetracementCalculationWorker")
  def add_calculated_columns(self, data: pandas.DataFrame):
    identifier_grouped_data: pandas.DataFrame = data.groupby(BaseColumns.Identifier)
    #Since, our dataframe may contain data for multiple symbols, we need to first group them by Identifier
    data[self._column_name] = identifier_grouped_data[BaseColumns.Close].transform(
        lambda x: 
          x.rolling(self._time_window).max() - 
          (
            (x.rolling(self._time_window).max() - x.rolling(self._time_window).min())  * self._level
          )
      )

In [25]:
# Create pipline with the FibnocciRetracementCalculationWorker and run 
from markets_insights.datareader.data_reader import NseIndicesReader, DateRangeCriteria
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor, HistoricalDataProcessOptions, \
  MultiDataCalculationPipelines, CalculationPipeline
histDataProcessor = HistoricalDataProcessor(HistoricalDataProcessOptions(include_monthly_data=False, include_annual_data=False))

# Fetch the data
result = histDataProcessor.process(NseIndicesReader(), DateRangeCriteria(datetime.date(2023, 12, 1), datetime.date(2023, 12, 31)))

# Prepare calculation pipeline
fbr50_worker = FibonacciRetracementCalculationWorker(time_window=7, level_perct=50)
pipelines = MultiDataCalculationPipelines()
histDataProcessor.set_calculation_pipelines(
  CalculationPipeline(
    workers = [fbr50_worker]
  )
)

# Run the pipeline and get data
histDataProcessor.run_calculation_pipelines()

Started to read data
HistoricalDataProcessor.get_data took 0 seconds
HistoricalDataProcessor.get_manual_data took 0 seconds
HistoricalDataProcessor.run_base_calculations took 0 seconds
HistoricalDataProcessor.process took 1 seconds
FibnocciRetracementCalculationWorker took 0 seconds


c:\Data\_Personal\Projects\markets_insights\src\markets_insights\dataprocess\data_processor.py:329: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([historical_data, manual_data], ignore_index=True)


In [26]:
## Display the results
print(result.get_daily_data()[[
  BaseColumns.Identifier, BaseColumns.Date, BaseColumns.Close, fbr50_worker._column_name
]].tail(5).to_markdown())

|      | Identifier                  | Date                |    Close |    Fbr50 |
|-----:|:----------------------------|:--------------------|---------:|---------:|
| 2141 | NIFTY50 USD                 | 2023-12-29 00:00:00 |  9048.9  |  8941.77 |
| 2142 | NIFTY 10 YR BENCHMARK G-SEC | 2023-12-29 00:00:00 |  2232.79 |  2230.24 |
| 2143 | NIFTY200 ALPHA 30           | 2023-12-29 00:00:00 | 20858.6  | 20358.2  |
| 2144 | NIFTY HEALTHCARE INDEX      | 2023-12-29 00:00:00 | 10637.8  | 10422.7  |
| 2145 | NIFTY AUTO                  | 2023-12-29 00:00:00 | 18618.2  | 18070.9  |
